In [18]:
import random as rand

In [19]:
# Variables importantes
numero_poblacion = 10
generaciones = 50
mochila = 10
peso_mochila = 30
probabilidad_cruza = 0.85
probabilidad_mutacion = 0.1
cruza_uniforme = 0.5

# Decoy detonators, love potion (min 3), extendable ears, skiving snackbox(min 2), fever fudge, puking pastilles, nosebleed nougat
ganancia = [10, 8, 12, 6, 3, 2, 2]
peso = [4, 2, 5, 5, 2, 1.5, 1]

In [20]:
def calcular_peso_individuo(individuo):
    peso_total = 0

    for i in range(7):
        peso_total += individuo[i] * peso[i]

    return peso_total

In [21]:
def calcular_peso_poblacion(poblacion):
    peso = []
    for individuo in poblacion:
        peso.append(calcular_peso_individuo(individuo))
    return peso

In [22]:
# Generar población inicial
def poblacion(individuos):
    
    poblacion = []
    j = 0
    
    for i in range (individuos):
        
        individuo = [0,0,0,0,0,0,0]
        
        while j <= 6:
            if j == 1:
                individuo[1] = rand.randint(3,10)
            elif j == 3:
                individuo[3] = rand.randint(2,10)
            else:
                individuo[j] = rand.randint(0,10)
            j += 1 
        poblacion.append(individuo)
        j = 0
    
    return poblacion

In [23]:
def aptitud(poblacion):
    suma_total = 0
    suma_indv = [0]*numero_poblacion
    j = 0

    for individuo in poblacion:
        for i in range(7):
            suma_total += individuo[i] * ganancia[i]
            suma_indv[j] += individuo[i] * ganancia[i]
        j += 1

    # Calcular aptitud
    aptitud = []
    aptitud_acumulada = []

    for i in range (len(poblacion)):
        aptitud.append(suma_indv[i]/suma_total)
        
        if i == 0:
            aptitud_acumulada.append(aptitud[i])
        else:
            aptitud_acumulada.append(aptitud_acumulada[i-1]+aptitud[i])

    return aptitud_acumulada

In [24]:
def aptitud_indv(individuo):
    suma_total = 0

    for i in range(7):
        suma_total += individuo[i] * ganancia[i]

    return suma_total

In [25]:
def mutacion(hijo1,hijo2):
    
    var_aleatoria1 = []
    var_aleatoria2 = []
    condicion = True

    for i in range(7):
        var_aleatoria1.append(rand.uniform(0,1))
        var_aleatoria2.append(rand.uniform(0,1))

    for i in range(7):
        if probabilidad_mutacion >= var_aleatoria1[i]:
            if i == 1:
                hijo1[i] = rand.randint(3,10)
            elif i == 3:
                hijo1[i] = rand.randint(2,10)
            else:
                hijo1[i] = rand.randint(0,10)
        if probabilidad_mutacion >= var_aleatoria2[i]:
            if i == 1:
                hijo2[i] = rand.randint(3,10)
            elif i == 3:
                hijo2[i] = rand.randint(2,10)
            else:
                hijo2[i] = rand.randint(0,10)
    
    if calcular_peso_individuo(hijo1) > peso_mochila or calcular_peso_individuo(hijo2) > peso_mochila:
        condicion = False

    return hijo1, hijo2, condicion

In [26]:
def cruza(padres):
    
    var_aleatoria = []
    hijo1 = []
    hijo2 = []
    
    for i in range(7):
        var_aleatoria.append(rand.uniform(0,1))
    
    for i in range(7):
        if cruza_uniforme >= var_aleatoria[i]:
            hijo1.append(padres[0][i])
            hijo2.append(padres[1][i])
        else:
            hijo1.append(padres[1][i])
            hijo2.append(padres[0][i])
    
    return mutacion(hijo1, hijo2)

In [27]:
def ruleta(aptitud, poblacion):
        
    padres = []
    random1 = rand.uniform(0,1)
    
    for i, apt in enumerate(aptitud):
        if apt >= random1 and poblacion[i] not in padres:
            padres.append(poblacion[i])
            break
            
    while len(padres) != 2:
        random2 = rand.uniform(0,1)
        for i, apt in enumerate(aptitud):
            if apt >= random2 and poblacion[i] not in padres:
                padres.append(poblacion[i])
                break
        
    random_cruza = rand.uniform(0,1)
    if probabilidad_cruza >= random_cruza:
        return padres, True
    else:
        return padres, False

In [28]:
def padre_mas_debil(hijos, padres):
    
    individuos = [(aptitud_indv(hijos[0]), hijos[0]),(aptitud_indv(hijos[1]), hijos[1]),(aptitud_indv(padres[0]), padres[0]),(aptitud_indv(padres[1]), padres[1])]
    
    mejor1 = max(individuos)
    individuos.remove(mejor1)
    mejor2 = max(individuos)
    
    return [mejor1[1], mejor2[1]]

In [ ]:

ind = 1

while ind:
    pob = poblacion(numero_poblacion)
    pesos = calcular_peso_poblacion(pob)
    for i in pesos:
        if i <= peso_mochila:
            ind = 0
        else:
            ind = 1

for i in range(generaciones):
    print("Generación: ", i+1)
    print("Población actual: ",pob)
    apt = aptitud(pob)
    x = 0
    nueva_poblacion = []
    while x < numero_poblacion/2:

        padres, flag = ruleta(apt, pob)

        if flag:
            hijo1, hijo2, flag2 = cruza(padres)
            while flag2 is False:
                padres, flag = ruleta(apt, pob)
                hijo1, hijo2, flag2 = cruza(padres) 
            hijos = [hijo1, hijo2]
            individuos_optimos = padre_mas_debil(hijos, padres)
            nueva_poblacion.append(individuos_optimos[0])
            nueva_poblacion.append(individuos_optimos[1])

        else:
            nueva_poblacion.append(padres[0])
            nueva_poblacion.append(padres[1])

        x += 1

    print("nueva_poblacion: ", nueva_poblacion)
    pob = nueva_poblacion

Generación:  1
Población actual:  [[9, 10, 3, 8, 2, 2, 6], [0, 10, 6, 3, 2, 0, 3], [4, 10, 9, 8, 3, 7, 10], [7, 8, 3, 10, 9, 5, 6], [0, 10, 8, 6, 4, 8, 6], [8, 7, 0, 5, 4, 2, 6], [1, 9, 1, 7, 9, 0, 3], [6, 9, 3, 9, 8, 7, 4], [0, 3, 7, 4, 7, 7, 8], [0, 4, 1, 2, 0, 0, 3]]
nueva_poblacion:  [[0, 10, 6, 3, 2, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 6, 0, 3, 0, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3]]
Generación:  2
Población actual:  [[0, 10, 6, 3, 2, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 6, 0, 3, 0, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3], [0, 10, 6, 3, 2, 0, 3], [0, 4, 1, 2, 2, 0, 3]]
nueva_poblacion:  [[0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 1, 2, 0, 0, 3], [0, 6, 0, 2, 2, 0, 3], [0